In [1]:
from os import listdir, mkdir
from os.path import isfile, join, exists

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
SAVE_IMAGE = False

# Save image as file
def save_convergence_image(fig, filename):
    if not exists("images/convergence"):
        mkdir("images/convergence")

    fig.write_image("images/convergence/" + filename + ".svg")
    fig.write_image("images/convergence/" + filename + ".png")

In [22]:
def filter_on_size(files, size):
    return list(filter(lambda f: size in f, files))

# "./foo/bar/n120m8-01.csv" -> "n120m8-01"
def extract_filename(filenames):
    return [f.split("/")[-1].split(".")[0] for f in filenames]

def get_files_from_folder(folder):
    return [join(folder, f) for f in listdir(folder) if isfile(join(folder, f))]

def get_all_results(files):
    results = []
    
    for file in files:
        df = pd.read_csv(
            file, header=0, names=["iterations", "makespan", "count", "time"], index_col=None
        )

        results.append(df)
        
    return results

# Calculate makespan at time intervals
def calculate_best_makespan(df, num_windows, window_size):
    makespan = [df.makespan.max()]

    for window_idx in range(1, num_windows):
        window = window_size * window_idx
    
        makespan_smaller_than = df[df["time"] <= window]
    
        makespan.append(makespan_smaller_than["makespan"].min())
        
    return makespan

def get_convergence_df(folder, num_windows, size="n120m8"):
    files = get_files_from_folder(folder)

    # Keep only relevant instance sizes
    files = filter_on_size(files, size)

    # Get list of DataFrames with all relevant results
    dfs = get_all_results(files)

    # Extract name of instance from path to result file for each instance
    files = extract_filename(files)

    # Calculate maximum time from time column
    max_time = dfs[0]["time"].max()

    # Define number of windows to divide results into and define window size
    window_size = max_time / num_windows

    # Create list of all increments in window size
    index = [round(window_size * i, 0) for i in range(num_windows)]

    res = []

    # Iterate all DataFrames and calculate best makespan at each time interval
    for df in dfs:
        makespan = calculate_best_makespan(df, num_windows, window_size)

        res.append(makespan)


    # Concatenate all results into one DataFrame
    df = pd.DataFrame(res, columns=index, index=files).transpose()

    df = df.transform(lambda x: round(x / x.max() * 100, 1))

    df.index = df.index / 1000
    
    return df

def calc_stats(dfs):
    stats = []
    for df in dfs:
        stats.append(df.agg(["mean", "sem"], axis=1))
        
    return stats


def plot_convergence(folders, num_windows=100, size="n120m8", names=["GA", "IG"]):
    dfs = []
    for folder in folders:
        dfs.append(get_convergence_df(folder, num_windows, size))
        
    
    stats = calc_stats(dfs)
    
    fig = go.Figure()
    for i, df in enumerate(stats):
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df["mean"],
                name=names[i],
            )
        )

    fig.update_layout(
            template="simple_white",
            xaxis_title="Time (s)",
            yaxis_title="Makespan (% of initial)",
            font_family="Times New Roman",
        )

    fig.update_xaxes(mirror=True)
    fig.update_yaxes(mirror=True)
    
    return fig

In [23]:
size = "n120m8"
fig = plot_convergence(
    ["../solutions/improvement/ga/all/", 
     "../solutions/improvement/ig/all/"],
    num_windows=100,
    size=size
)

fig.show()

if SAVE_IMAGE:
    save_convergence_image(fig, size)

In [24]:
size = "n80m8"
fig = plot_convergence(
    ["../solutions/improvement/ga/all/"],
    num_windows=100,
    size=size
)

fig.show()

if SAVE_IMAGE:
    save_convergence_image(fig, size)